In [0]:
dbutils.fs.unmount("/mnt/azuredatabricksadolfo/bronze")
dbutils.fs.mount (
    source = "wasbs://bronze@azuredatabricksadolfo.blob.core.windows.net", 
   mount_point = "/mnt/azuredatabricksadolfo/bronze", 
    extra_configs = {"fs.azure.account.key.azuredatabricksadolfo.blob.core.windows.net":"qPxqIfzzU3T5qh6rYJva3bkP4axJQ57KwssIWpsW31Ttcvg6WhDWictW7EvEDytFHwXlGUpWYvTO+AStw/BwgQ=="}
                  )

True

In [0]:
dbutils.fs.mount (source = "wasbs://silver@azuredatabricksadolfo.blob.core.windows.net", # origem do arquivo
                  mount_point = "/mnt/azuredatabricksadolfo/silver", # localização do arquivo
                  extra_configs = {"fs.azure.account.key.azuredatabricksadolfo.blob.core.windows.net":"qPxqIfzzU3T5qh6rYJva3bkP4axJQ57KwssIWpsW31Ttcvg6WhDWictW7EvEDytFHwXlGUpWYvTO+AStw/BwgQ=="} # chave de acesso ao blob storage

  File <command-7593693771494678>, line 3
    extra_configs = {"fs.azure.account.key.azuredatabricksadolfo.blob.core.windows.net":"qPxqIfzzU3T5qh6rYJva3bkP4axJQ57KwssIWpsW31Ttcvg6WhDWictW7EvEDytFHwXlGUpWYvTO+AStw/BwgQ=="} # chave de acesso ao blob storage
                                                                                                                                                                                                                     ^
SyntaxError: incomplete input


In [0]:

dbutils.fs.mount (source = "wasbs://gold@azuredatabricksadolfo.blob.core.windows.net", # origem do arquivo
                  mount_point = "/mnt/azuredatabricksadolfo/gold", # localização do arquivo
                  extra_configs = {"fs.azure.account.key.azuredatabricksadolfo.blob.core.windows.net":"qPxqIfzzU3T5qh6rYJva3bkP4axJQ57KwssIWpsW31Ttcvg6WhDWictW7EvEDytFHwXlGUpWYvTO+AStw/BwgQ=="}) # chave de acesso ao blob storage

In [0]:
#criar database

spark.sql("CREATE DATABASE IF NOT EXISTS covid")

In [0]:
# ler csv da camada bronze
df_covid_br_bronze = spark.read.format('csv').options(header = 'true', inferSchema = 'true', delimiter = ';').load('dbfs:/mnt/azuredatabricksadolfo/bronze/SRAG_01-06.csv')

In [0]:
#limpeza dados vazios
df_covid_silver = df_covid_br_bronze.filter(df_covid_br_bronze.ID_REGIONA.isNotNull)


In [0]:
#CRIAR ARQUIVO PARQUET NO DIRETÓRIO
# BUSCAR A INFORMAÇÃO LIMPA E JOGAR NOVAMENTE NO DIRETÓRIO NO FORMATO PARQUET
df_covid_silver.write.format('delta').mode('overwrite').save('/mnt/azuredatabricksadolfo/silver/br_covid')


In [0]:
display(dbutils.fs.mounts())

In [0]:
# Garanta que a estrutura Delta existe lê o conteúdo do arquivo

display(spark.read.format("delta").load("/mnt/azuredatabricksadolfo/silver/br_covid"))

In [0]:
df = spark.read.format("delta").load("/mnt/azuredatabricksadolfo/silver/br_covid")

In [0]:
# cria o schema e depois cria a tabela mencionada dentro do schema
# vai criar dentro do pipelinedatabricks o schema silver e depois a tabela mencionada

spark.sql("CREATE SCHEMA IF NOT EXISTS silver")
df.write.mode("overwrite").saveAsTable("silver.br_covid")

In [0]:

# query para verificar os dados na tabela br_covid

br_covid = spark.sql('SELECT * FROM silver.br_covid')

display(br_covid)

In [0]:
# início dos trabalhos na camada Gold
from pyspark.sql.functions import concat, to_date , year, month 

In [0]:
# data frame para inicializar os trabalhos na camada gold
# armazenar em um novo dataframe os dados do arquivo br_covid

df_covid_br_silver = spark.read.format('delta').load('/mnt/azuredatabricksadolfo/silver/br_covid')

In [0]:
df_covid_br_silver = df_covid_br_silver.withColumn('DT_NOTIFIC', to_date(df_covid_br_silver['DT_NOTIFIC']))
df_covid_br_silver = df_covid_br_silver.withColumn('DT_SIN_PRI', to_date(df_covid_br_silver['DT_SIN_PRI']))
df_covid_br_silver = df_covid_br_silver.withColumn('DT_NASC', to_date(df_covid_br_silver['DT_NASC']))


In [0]:
# CRIAR UMA NOVA COLUNA NO DATAFRAME, PARA ARMAZENAR A DATA DA CARGA DO ARQUIVO
# ESTA COLUNA SERÁ GRAVADA SOBRE AS ALTERAÇÕES JÁ REALIZADAS NO DATAFRAME ANTERIOR, POR ISSO SERÁ NECESSÁRIO CRIAR UM DATAFRAME NOVO
#

from pyspark.sql.functions import current_date
df_covid_br_gold = df_covid_br_silver.withColumn('DT_CARGA', current_date())

In [0]:
# RENOMEAR ALGUMAS COLUNAS DO DATAFRAME, PARA MELHORAR A LEGIBILIDADE DOS DADOS 

df_covid_br_gold = df_covid_br_gold.withColumnRenamed('DT_NOTIFIC', 'DT_NOTIFICACAO').withColumnRenamed('DT_SIN_PRI', 'DT_PRIMEIROS_SINTOMAS').withColumnRenamed('DT_NASC', 'DT_NASCIMENTO')

In [0]:
# ACRESCENTAR MAIS DUAS COLUNAS NO DATAFRAME PARA ARMAZENAR EM SEPARADO AS INFORMAÇÕES DO MES E DO ANO 'YEAR' E 'MONTH'

df_covid_br_gold = df_covid_br_gold.withColumn('YEAR', year(df_covid_br_gold['DT_NOTIFICACAO']))
df_covid_br_gold = df_covid_br_gold.withColumn('MONTH', month(df_covid_br_gold['DT_NOTIFICACAO']))
df_covid_br_gold = df_covid_br_gold.withColumn('YEAR', year(df_covid_br_gold['DT_PRIMEIROS_SINTOMAS']))
df_covid_br_gold = df_covid_br_gold.withColumn('MONTH', month(df_covid_br_gold['DT_PRIMEIROS_SINTOMAS']))
df_covid_br_gold = df_covid_br_gold.withColumn('YEAR', year(df_covid_br_gold['DT_NASCIMENTO']))
df_covid_br_gold = df_covid_br_gold.withColumn('MONTH', month(df_covid_br_gold['DT_NASCIMENTO']))



In [0]:
#particiona as informações do arquivo por ano e mês para facilitar a busca posterior e eventual gravação em banco de dados

df_covid_br_gold.write.format('delta').mode('overwrite').option("mergeSchema","true").partitionBy('YEAR', 'MONTH').save('/mnt/azuredatabricksadolfo/gold/br_covid_gold')